In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

2022-12-13 18:20:23.625299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
image_size = 64

train_dir = './dataset/'

def get_data(dir):
    X = []
    y = []
    ctr = 0
    for cat in os.listdir(dir):
        for img_file in os.listdir(dir + cat + '/'):
            label = cat
            img = cv2.imread(dir + cat + '/' + img_file)
            img = cv2.resize(img, (image_size, image_size))
            img_arr = np.asarray(img).reshape((image_size, image_size, 3))

            X.append(img_arr)
            y.append(label)

            if ctr % 1000 == 0:
                print(str(ctr)+'/87000')
            ctr += 1
    
    return X,y

X_train, y_train = get_data(train_dir)


0/87000
1000/87000
2000/87000
3000/87000
4000/87000
5000/87000
6000/87000
7000/87000
8000/87000
9000/87000
10000/87000
11000/87000
12000/87000


In [8]:
# Convert to np array
X_train = np.array(X_train, dtype=np.float64)
y_train = np.array(y_train)

9410


In [5]:
X_data = X_train
y_data = y_train

#split data into test/ train set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=69, stratify=y_data)

In [9]:
# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# fit the labels -> [ a , b, c, d ... ] -> [0, 1, 2 ,3 ..... ]
# encode the label -> [ m, j ,c , ...] -> [16, 7, 3, .... ]
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

from tensorflow.keras.utils import to_categorical
y_train_onehot = to_categorical(y_train_encoded, num_classes=25)
y_test_onehot = to_categorical(y_test_encoded, num_classes=25)


In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train_encoded.shape)
print(y_test_encoded.shape)

(9410, 64, 64, 3)
(9410,)
(3137, 64, 64, 3)
(3137,)
(9410,)
(3137,)


In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(32, (5,5), input_shape=(64,64,3), data_format='channels_last'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3)))

model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(25, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 60, 60, 32)        2432      
                                                                 
 activation_3 (Activation)   (None, 60, 60, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 activation_4 (Activation)   (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                  

In [16]:
model.fit(X_train, y_train_onehot,
          epochs=50,
          batch_size=64,
          verbose=1,
          validation_data=(X_test, y_test_onehot),
          callbacks=[early_stop])

Epoch 1/50
148/148 [==============================] - 28s 180ms/step - loss: 1.9061 - accuracy: 0.6712 - val_loss: 0.5635 - val_accuracy: 0.8177
Epoch 2/50
148/148 [==============================] - 31s 210ms/step - loss: 0.1417 - accuracy: 0.9562 - val_loss: 0.1324 - val_accuracy: 0.9519
Epoch 3/50
148/148 [==============================] - 31s 209ms/step - loss: 0.0321 - accuracy: 0.9904 - val_loss: 0.0633 - val_accuracy: 0.9780
Epoch 4/50
148/148 [==============================] - 27s 182ms/step - loss: 0.0173 - accuracy: 0.9947 - val_loss: 0.0511 - val_accuracy: 0.9825
Epoch 5/50
148/148 [==============================] - 28s 189ms/step - loss: 0.0120 - accuracy: 0.9967 - val_loss: 0.0291 - val_accuracy: 0.9898
Epoch 6/50
148/148 [==============================] - 27s 181ms/step - loss: 0.0083 - accuracy: 0.9982 - val_loss: 0.0422 - val_accuracy: 0.9895
Epoch 7/50
148/148 [==============================] - 26s 174ms/step - loss: 0.0011 - accuracy: 0.9999 - val_loss: 0.0260 - val_ac

In [17]:
# save model
model.save('./grm')

INFO:tensorflow:Assets written to: ./grm/assets


INFO:tensorflow:Assets written to: ./grm/assets
